In [11]:
import numpy as np, pandas as pd, time, re
from datetime import datetime
import os
from datetime import datetime
from copy import deepcopy

In [12]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [13]:
from src.data_pipeline.DataLoader import DataLoader
from src.utility.sys_utils import get_spark

# Read data from files

In [14]:
# change the config file to your own file, otherwise it will not find proper path
dataloader = DataLoader(dataset_name="user_10_item_1_exp", config_name="default_config.json")
spark = get_spark(cores=4)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46207)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46207)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-110f27ce52f8>", line 3, in <module>
    spark = get_spark(cores=4)
  File "/home/ds2019/jianglin/Final Project/Personalization_Final/src/utility/sys_utils.py", line 16, in get_spark
    .config("spark.local.dir", local_dir)
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/sql/session.py", line 173, in getOrCreate
    sc = SparkContext.getOrCreate(sparkConf)
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/context.py", line 367, in getOrCreate
    SparkContext(conf=conf or SparkConf())
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/context.py", line 136, in __init__
    conf, jsc, profiler_cl

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46207)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-110f27ce52f8>", line 3, in <module>
    spark = get_spark(cores=4)
  File "/home/ds2019/jianglin/Final Project/Personalization_Final/src/utility/sys_utils.py", line 16, in get_spark
    .config("spark.local.dir", local_dir)
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/sql/session.py", line 173, in getOrCreate
    sc = SparkContext.getOrCreate(sparkConf)
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/context.py", line 367, in getOrCreate
    SparkContext(conf=conf or SparkConf())
  File "/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/pyspark/context.py", line 136, in __init__
    conf, jsc, profiler_cl

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

# Display side info schema

In [12]:
train_df = dataloader.get_train_set()

In [13]:
train_df.head()

,user_id,business_id,stars
0,-3i9bhfvrM3F1wsC9XIB8g,pSQFynH1VxkfSmehRXlZWw,5.0
1,-3i9bhfvrM3F1wsC9XIB8g,ZGU2--DiLxf7enO3I0mJxQ,3.0
2,-3i9bhfvrM3F1wsC9XIB8g,bvow46stEejfxnnqEKTAyQ,2.0
3,-3i9bhfvrM3F1wsC9XIB8g,PDIh1lVtKlUqPJROXYJQvQ,4.0
4,-3i9bhfvrM3F1wsC9XIB8g,GtHu9uGXpn7Jg_Z7vFKZKw,4.0


In [5]:
user_side_info = dataloader.get_user_side_info()

In [6]:
user_side_info.head()

,average_stars,cool,fans,funny,review_count,useful,user_id,cnt_compliment,yelp_days,elite_years
0,4.03,25,5,17,95,84,l6BmjZMeQD3rDxWUbiAiow,8,2275,3
1,3.63,16,4,22,33,48,4XChL029mKr5hydo79Ljxg,3,2504,0
2,3.71,10,0,8,16,28,bc8C_eETBWL0olvFSJJd0w,1,2279,0
3,4.85,14,5,4,17,30,dD0gZpBctWGdWo9WlGuhlA,4,2049,0
4,4.08,665,39,279,361,1114,MM4RJAeH6yuaN8oZDSt0RA,293,2260,4


In [7]:
item_side_info = dataloader.get_item_side_info()

In [8]:
item_side_info.head()

,business_id,top_category,review_count,stars,state,city
0,1SWheh84yJXfytovILXOAQ,Active Life,5,3.0,AZ,Phoenix
1,QXAEGFB4oINsVuTFxEYKFQ,Food,128,2.5,ON,Mississauga
2,gnKjwL_1w79qoiV3IC_xQQ,Restaurants,170,4.0,NC,Charlotte
3,xvX2CttrVhyG2z1dFg_0xw,Financial Services,3,5.0,AZ,Goodyear
4,HhyxOkGAM07SRYtlQ4wMFQ,Shopping,4,4.0,NC,Charlotte


# Fit with side information

In [8]:
from src.model.CollectiveMF import CollectiveMF
from src.data_pipeline.pipeline import test_evaluation

In [10]:
for use_user, use_item in [(True, False), (False, True), (False, False), (True, True)]:
    for k in [4,8,16]:
        for w_main in [0.5, 0.8, 1.0]:
            for reg in [1e-3, 1e-4]:
                param_dict = {
                    "w_main": w_main,
                    "use_user_info": use_user,
                    "use_item_info": use_item,
                    "k": k,
                    "reg_param": reg
                }
                cmf = CollectiveMF(params=param_dict)
                test_evaluation(data_loader=dataloader, model=cmf, spark=spark, caching=True)
#                 print(cmf.get_name())
                
                


Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.001-w_main_0.5/Ranking_fold_-1.parquet
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.001-w_main_0.5/Rating_fold_-1.parquet
Finish inference phase
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.0001-w_main_0.5/Ranking_fold_-1.parquet
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.0001-w_main_0.5/Rating_fold_-1.parquet
Finish inference phase





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functi

/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/cmfrec/__init__.py:1324: RuntimeWarning: invalid value encountered in add
  out += self.global_mean


Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 652.895874
  Number of iterations: 182
  Number of functions evaluations: 221
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_4-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4663.111328
  Number of iterations: 112
  Number of functions evaluations: 147
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_8-reg_param_0.001-w_main_0.5/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization 

/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/cmfrec/__init__.py:1324: RuntimeWarning: invalid value encountered in add
  out += self.global_mean


Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_16-reg_param_0.001-w_main_0.8/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 472.230469
  Number of iterations: 307
  Number of functions evaluations: 345
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_16-reg_param_0.0001-w_main_0.8/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4663.932617
  Number of iterations: 98
  Number of functions evaluations: 125
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_User/k_16-reg_param_0.001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimizatio

Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 6.857600
  Number of iterations: 261
  Number of functions evaluations: 300
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Item/k_16-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.847749
  Number of iterations: 6
  Number of functions evaluations: 12
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_No/k_4-reg_param_0.001-w_main_0.5/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.811203
  Number of iteration

/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/cmfrec/__init__.py:1324: RuntimeWarning: invalid value encountered in add
  out += self.global_mean


Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_No/k_16-reg_param_0.001-w_main_0.8/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 1.273432
  Number of iterations: 32
  Number of functions evaluations: 41
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_No/k_16-reg_param_0.0001-w_main_0.8/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1.683881
  Number of iterations: 10
  Number of functions evaluations: 33
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_No/k_16-reg_param_0.001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated 

/opt/anaconda3/envs/Spark/lib/python3.6/site-packages/cmfrec/__init__.py:1324: RuntimeWarning: invalid value encountered in add
  out += self.global_mean


Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_No/k_16-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4725.915527
  Number of iterations: 135
  Number of functions evaluations: 167
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_4-reg_param_0.001-w_main_0.5/Rating_fold_-1.parquet
Finish inference phase
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 584.150208
  Number of iterations: 243
  Number of functions evaluations: 271
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_4-reg_param_0.0001-w_main_0.5/Rating_fold_-1.parquet
Finish infe

# Calculate top-k recommendation ranking metric

In [2]:
# This part focuses on calculating NDCG@K and Precision@K result for the optimal model
# Note that k may vary from 1 to 10 in our settings

In [6]:
optimal_params = dict([('k', 16), ('reg_param', 0.0001), ('w_main', 1.0), ("use_user_info", True), ("use_item_info", True)])

In [16]:
cmf = CollectiveMF(params=optimal_params)
for top_k in range(10):
    test_evaluation(data_loader=dataloader, model=cmf, spark=spark, caching=True, top_k=top_k+1)

Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Ranking_fold_-1.parquet
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Ranking_fold_-1.parquet
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Ranking_fold_-1.parquet
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase
Using cached file from /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF_Both/k_16-reg_param_0.0001-w_main_1.0